# Siddharth Nagepalli - 210968074
# AI lab3 Exercise

In [1]:
pip install easyAI

Note: you may need to restart the kernel to use updated packages.


In [2]:
import easyAI

In [3]:
dir(easyAI)

['AI',
 'AI_Player',
 'DUAL',
 'DictTranspositionTable',
 'HashTranspositionTable',
 'Human_Player',
 'Negamax',
 'NonRecursiveNegamax',
 'Player',
 'SSS',
 'TranspositionTable',
 'TwoPlayerGame',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'mtd',
 'solve_with_depth_first_search',
 'solve_with_iterative_deepening']

In [26]:
from easyAI import TwoPlayerGame
from easyAI.Player import Human_Player


class TicTacToe(TwoPlayerGame):
    """The board positions are numbered as follows:
    1 2 3
    4 5 6
    7 8 9
    """

    def __init__(self, players):
        self.players = players
        self.board = [0 for i in range(9)]
        self.current_player = 1  # player 1 starts.

    def possible_moves(self):
        return [i + 1 for i, e in enumerate(self.board) if e == 0]

    def make_move(self, move):
        self.board[int(move) - 1] = self.current_player

    def unmake_move(self, move):  # optional method (speeds up the AI)
        self.board[int(move) - 1] = 0

    def lose(self):
        """ Has the opponent "three in line ?" """
        return any(
            [
                all([(self.board[c - 1] == self.opponent_index) for c in line])
                for line in [
                    [1, 2, 3],
                    [4, 5, 6],
                    [7, 8, 9],  # horiz.
                    [1, 4, 7],
                    [2, 5, 8],
                    [3, 6, 9],  # vertical
                    [1, 5, 9],
                    [3, 5, 7],
                ]
            ]
        )  # diagonal

    def is_over(self):
        return (self.possible_moves() == []) or self.lose()

    def show(self):
        print(
            "\n"
            + "\n".join(
                [
                    " ".join([[".", "O", "X"][self.board[3 * j + i]] for i in range(3)])
                    for j in range(3)
                ]
            )
        )

    def scoring(self):
        return -100 if self.lose() else 0


if __name__ == "__main__":

    from easyAI import AI_Player, Negamax

    ai_algo = Negamax(6)
    game=TicTacToe([Human_Player(), AI_Player(ai_algo)]).play()


. . .
. . .
. . .

Player 1 what do you play ? 5

Move #1: player 1 plays 5 :

. . .
. O .
. . .

Move #2: player 2 plays 1 :

X . .
. O .
. . .

Player 1 what do you play ? 7

Move #3: player 1 plays 7 :

X . .
. O .
O . .

Move #4: player 2 plays 3 :

X . X
. O .
O . .

Player 1 what do you play ? 2

Move #5: player 1 plays 2 :

X O X
. O .
O . .

Move #6: player 2 plays 8 :

X O X
. O .
O X .

Player 1 what do you play ? 6

Move #7: player 1 plays 6 :

X O X
. O O
O X .

Move #8: player 2 plays 4 :

X O X
X O O
O X .

Player 1 what do you play ? 9

Move #9: player 1 plays 9 :

X O X
X O O
O X O


In [31]:
from easyAI.AI import Negamax
from easyAI.Player import AI_Player


def solve_with_iterative_deepening(game, ai_depths, win_score, scoring=None, tt=None, verbose=True):
    
    
    
    if game.players is None:
        game.players = [AI_Player(None), AI_Player(None)]

    for depth in ai_depths:
        ai = Negamax(depth, scoring, tt=tt)
        ai(game)
        alpha = ai.alpha
        if verbose:
            print("d:%d, a:%d, m:%s" % (depth, alpha, str(game.ai_move)))
        if abs(alpha) >= win_score:
            break

    # 1:win, 0:draw, -1:defeat
    result = +1 if alpha >= win_score else (-1 if alpha <= -win_score else 0)

    return result, depth, game.ai_move




In [32]:
def solve_with_depth_first_search(game, win_score, maxdepth=50, tt=None, depth=0):
    
    

    # Is there a transposition table and is this game in it ?
    lookup = None if (tt is None) else tt.lookup(game)
    if lookup is not None:
        return lookup["value"]

    if depth == maxdepth:
        raise "Max recursion depth reached :("

    if game.is_over():
        score = game.scoring()
        value = 1 if (score >= win_score) else (-1 if -score >= win_score else 0)
        if tt is not None:
            tt.store(game=game, value=value, move=None)
        return value

    possible_moves = game.possible_moves()

    state = game
    unmake_move = hasattr(state, "unmake_move")

    best_value, best_move = -1, None

    for move in possible_moves:

        if not unmake_move:
            game = state.copy()  # re-initialize move

        game.make_move(move)
        game.switch_player()

        move_value = -solve_with_depth_first_search(
            game, win_score, maxdepth, tt, depth + 1
        )

        if unmake_move:
            game.switch_player()
            game.unmake_move(move)

        if move_value == 1:
            if tt is not None:
                tt.store(game=state, value=1, move=move)
            return move_value

        if move_value == 0 and best_value == -1:
            # Is forcing a draw possible ?
            best_value = 0
            best_move = move
    if tt is not None:
        tt.store(game=state, value=best_value, move=best_move)

    return best_value

In [38]:
# creating an instance of the game
game_instance = TicTacToe([Human_Player(), AI_Player(None)])

In [48]:
ai_depths = [10]
win_score = 0
# Call the function
result, depth, move = solve_with_iterative_deepening(game_instance, ai_depths, win_score)

# Print the result using iterative deeping method 
print("Result:", result)
print("Depth reached:", depth)
print("Move:", move)

d:10, a:0, m:1
Result: 1
Depth reached: 10
Move: 1


In [49]:
best_value=solve_with_depth_first_search(game_instance,win_score)
# print the best result using depth first search
print(best_value)

-1


In [ ]:
# In iterative deepening, it determines if by playing perfectly the first player can force a win,
# or whether it will always lose against a perfect opponent.
# Using Iterative deepening we get the value 1 indicating victory for the first player. ( depth of 10 is reached )
#This algorithm explores the game by using several times the Negamax algorithm, always starting at the initial state of the game, 
# but taking increasing depth (in the list ai_depths) until the score of the initial condition indicates that the first player 
# will certainly win or loose, in which case it stops



# In depth-first search, the game is explored until endgames are reached.The endgames are evaluated to see if there are victories 
# or defeats. 
# Using depth first search we get the value -1 ( indicating defeat)
# This algorithm can be faster but less informative than id_solve, as it does not provide ‘optimal’ strategies (like shortest path to the victory). 
# It returns simply 1, 0, or -1 to indicate certain victory, draw, or defeat of the first player.